In [ ]:
include("../src/ECM_TDA.jl")
using .ECM_TDA
using Eirene
using Ripserer
using CSV
using TiffImages
using Images
using NPZ
using Plots
using Distances
using Clustering
using PersistenceDiagrams
using Measures
using Distributions
using MultivariateStats
using LinearAlgebra
using Random
using StatsBase
using JLD2
using FileIO
using PersistenceDiagrams
using DelimitedFiles
using UMAP
using DataFrames

In [ ]:
function PI_to_PCA2(PI_dict; pratio = 0.99)
    
    # subtract the mean
    n = length(PI_dict)
    PI_array = hcat([vec(PI_dict[i]) for i =1:n]...)
    PI_centered = PI_array .- mean(PI_array, dims = 2)
    
    # variance explained with 1 component
    M = fit(PCA, PI_centered, maxoutdim = 1)
    transformed = MultivariateStats.transform(M, PI_centered)
    variance_1 = principalratio(M)
    
    # variance explained with 2 components
    M = fit(PCA, PI_centered, maxoutdim = 2)
    transformed = MultivariateStats.transform(M, PI_centered)
    variance_2 = principalratio(M)

    # variance explained with 4 components
    M = fit(PCA, PI_centered, maxoutdim = 4)
    transformed = MultivariateStats.transform(M, PI_centered)
    variance_4 = principalratio(M)
    
    # perform PCA
    M = fit(PCA, PI_centered, pratio = pratio)
    transformed = MultivariateStats.transform(M, PI_centered)
    
    # get eigenvectors
    n_eigenvectors = size(transformed, 1)
    eigenvectors_array = projection(M)
    eigenvectors = Dict(i => reshape(eigenvectors_array[:,i], 20,20) for i = 1:n_eigenvectors)
    
    return transformed, eigenvectors, variance_1, variance_2, variance_4
end

In [ ]:
# color palettes

c1 = "#fd5184" # pink
c2 = "#ffb602" # yellow
c3 = "#3ec0c9" # blue / teal 
c4 = "#926EAE" # light purple
c5 = "#49a849" # green
c6 = "#F28522" # orange
c7 = "#265BF5" # dark blue 
c8 = "#AEF359" # lime green
c9 = "#FF1DCE" # purple 


# assign colors to cell types
c_cancer = "#2EC4B6" # light blue
c_leukocytes = "#919090" # grey 
c_ECM = "#EF476F" # pink
gr()

In [ ]:
function plot_ECM_PSRH(group_selected,
    idx_files,
    save_name;
    ECM_dir = "data/4000x4000_combined/subregion_ECM/",
    PSRH_dir = "data/4000x4000_combined/PSRH/",
    grid_layout = nothing,
    size = nothing,
    right_margin = 4mm,
    left_margin = -7mm,
    bottom_margin = 0mm)
    plot_array = []
    n_group = length(group_selected)
    n_ROI = length(group_selected[1])
    for i=1:n_group
        R = group_selected[i]
        for idx in R
            f = idx_files[idx]

            p_ECM = Images.load(ECM_dir * f * ".tif" )
            p_PSRH = Images.load(PSRH_dir * f * ".tif")
            push!(plot_array, plot(p_ECM, ticks = [], frame = :box))
            push!(plot_array, plot(p_PSRH, ticks = [], frame = :box, left_margin = left_margin, right_margin = right_margin, bottom_margin = bottom_margin))
        end
    end

    if grid_layout == nothing
        grid_layout = grid(n_group, n_ROI * 2)
    end

    if size == nothing
        size = (250 * n_ROI * 2, 250 * n_group)
    end
    p = plot(plot_array..., layout = grid_layout, size = size)
    savefig(save_name)
end
# get indices with large and small i-th coordinates
function get_coordinate_min_max_examples(transformed, i; n =4)

    sorted = sortperm(transformed[i,:])
    min_indices = sorted[1:n]
    max_indices = sorted[end-n+1:end]
    return min_indices, max_indices
end

function print_cluster_sizes(hc_clusters)
    clusters = unique(hc_clusters)
    for i in clusters
       println("size of cluster " * string(i) * ": ", count(x -> x == i, hc_clusters)) 
    end
end


# Create PH feature vectors

In [ ]:
# load PD
PD = load("data/4000x4000_combined/ECM_PD/PD.jld2")
PD0 = PD["PD0_ECM"]
PD1 = PD["PD1_ECM"]

# all subregion centers
subregion_centers = load("data/4000x4000/subregion_centers.jld2")["subregion_centers"];
subregion_centers_green = load("data/4000x4000_201222/subregion_centers_green.jld2")["subregion_centers_green"]
subregion_centers_purple = load("data/4000x4000_201222/subregion_centers_purple.jld2")["subregion_centers_purple"]
subregion_all = merge(subregion_centers, subregion_centers_green, subregion_centers_purple);

In [ ]:
# recompute coarser PI
PH0_dict = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD0 if v != nothing);
PH1_dict = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD1 if v != nothing);

PI0 = PersistenceImage([PH0_dict[k] for k in keys(PH0_dict)], sigma=50, size = 20)
PI1 = PersistenceImage([PH1_dict[k] for k in keys(PH1_dict)], sigma=50, size = 20)


ECM_PI0 = Dict()
for i in keys(PH0_dict)
    ECM_PI0[i] = PI0(PH0_dict[i])
end

ECM_PI1 = Dict()
for i in keys(PH1_dict)
    ECM_PI1[i] = PI1(PH1_dict[i])
end


In [ ]:
# save the min, max coordinates of PDs (useful for plotting)
PI0_xmin = PI0.xs[1]
PI0_xmax = PI0.xs[end]
PI0_ymin = PI0.ys[1]
PI0_ymax = PI0.ys[end]

PI1_xmin = PI1.xs[1]
PI1_xmax = PI1.xs[end]
PI1_ymin = PI1.ys[1]
PI1_ymax = PI1.ys[end]

In [ ]:
# save("data/4000x4000_combined/ECM_PD/PI_ranges.jld2",
#     "PI0_xmin", PI0_xmin,
#     "PI0_xmax", PI0_xmax,
#     "PI0_ymin", PI0_ymin,
#     "PI0_ymax", PI0_ymax,
#     "PI1_xmin", PI1_xmin,
#     "PI1_xmax", PI1_xmax,
#     "PI1_ymin", PI1_ymin,
#     "PI1_ymax", PI1_ymax)

In [ ]:
idx_ROI = load("data/4000x4000_combined/ECM_PI01_idx_files.jld2")["idx_files"];

In [ ]:
# plot example PI

i = 252
k = idx_ROI[i]
p1 = plot_PD(PD1[k])
p2 = heatmap(ECM_PI1[k])
plot(p1, p2, size = (500, 200))

# ECM analysis - using combined dim-0 and dim-1 PH features
* The code in this notebook shows the initial versions of UMAP & clustering.
* see notebook "6b_ECM_UMAP_clustering.ipynb" for the final version.

In [ ]:
# combine features
features = Dict()
for f in keys(ECM_PI0)
    # check that f is a key in all dictionaries
    if f in keys(ECM_PI1)
        combined = vcat(ECM_PI0[f], vec(ECM_PI1[f]))
        features[f] = combined
    end
end

In [ ]:
# index and ROIs 
#ROIs = collect(keys(features))
#idx_ROI = Dict(i => roi for (i, roi) in enumerate(ROIs));

#save("data/4000x4000_combined/ECM_PI01_idx_files.jld2", "idx_files", idx_ROI)
idx_ROI = load("data/4000x4000_combined/ECM_PI01_idx_files.jld2")["idx_files"];

In [ ]:
# prepare features array
n = length(idx_ROI)
features_array = hcat([features[idx_ROI[i]] for i = 1:n]...)
println("features array shape: ", size(features_array))

features_centered = features_array .- mean(features_array, dims = 2);

In [ ]:
#save("analysis/ECM/combined/features.jld2", "features", features_array)

### Umap

In [ ]:
# compute UMAP & save
#embedding = umap(features_centered, 2; n_neighbors = 5);
#writedlm("analysis/ECM/combined/umap.csv", embedding, ",")

# load
embedding = Array(CSV.read("analysis/ECM/combined/umap.csv", header = false))

In [ ]:
gr()
n = size(embedding, 2)
p = scatter(embedding[1,:], embedding[2,:], 
        markercolor = "slategrey",
        markersize = 5, 
        label = "", 
        xticks = [], 
        yticks = [], 
        framestyle = :box,  
        xlabel = "UMAP-1",
        ylabel = "UMAP-2",
        guidefontsize = 15,
        leftmargin = 5mm,
        size = (450, 350),
        hover = 1:n,
        legend = :topright)

# Clustering - hierarchical 

In [ ]:
# prepare features array
n = length(idx_ROI)
features_array = hcat([features[idx_ROI[i]] for i = 1:n]...)
println("features array shape: ", size(features_array))

features_centered = features_array .- mean(features_array, dims = 2);
dimred_embedding = umap(features_centered, 2; n_neighbors = 10, min_dist = 0.00001);
println("size of reduced dimension embedding: ", size(dimred_embedding))

d = Distances.pairwise(Euclidean(), dimred_embedding, dims = 2)
println("distance matrix shape: ", size(d))

hc = hclust(d);

In [ ]:
plot(hc)

In [ ]:
h_clusters = cutree(hc, k =9);
print_cluster_sizes(h_clusters)

In [ ]:
cluster_indices = Dict(i => findall(x -> x == i, h_clusters) for i in unique(h_clusters));

In [ ]:
# reorder clusters
c_reordered = Dict(1 => cluster_indices[1],
                   2 => cluster_indices[7],
                   3 => cluster_indices[2],
                   4 => cluster_indices[5],
                   5 => cluster_indices[8],
                   6 => cluster_indices[6],
                   7 => cluster_indices[3],
                   8 => cluster_indices[4],
                   9 => cluster_indices[9],)
cluster_indices = c_reordered;

In [ ]:
function plot_dim_red2(y, groups; 
    dim_red = "UMAP", 
    xaxis = "UMAP-1", 
    yaxis = "UMAP-2", 
    kwargs...)
    """
    groups: Dictionary of form (i => [indices])
    """

    ### specify colors ###
    c = Dict(
    0 => :grey,
    1 => "#fd5184", # pink
    2=> "#F28522", # orange
    3 => "#ffb602", # yellow
    4 => "#AEF359", # lime green
    5 => "#49a849", # green
    6 => "#3ec0c9", # blue / teal 
    7 => "#265BF5", # dark blue 
    8 => "#8B008B", # purple 
    9 => "#215467", # ocean
    )

    ### marker shapes ###
    markershapes = Dict(1 => :rect,
            2 => :utriangle,
            3 => :star,
            4 => :pentagon,
            5 => :diamond,
            6 => :dtriangle,
            7 => :star8,
            8 => :octagon,
            9 => :star4,
            10 => :pentagon
            )
    ### label
    if dim_red == "UMAP"
        label_prefix = "U"
    else
        label_prefix = "R"
    end

    # other figure parameters
    markersize = 3
    legendfontsize = 5
    markerstrokewidth = 1

    n = size(y, 2)
    n_groups = length(groups)

    annotated = vcat([val for (key, val) in groups]...)
    nonannotated = [i for i = 1:size(y, 2) if i ∉ annotated]
    p = scatter(y[1,nonannotated], y[2,nonannotated];
                markercolor = "lightgray",
                alpha = 0.6,
                markersize = markersize, 
                markerstrokewidth = markerstrokewidth,
                label = "", 
                xaxis = xaxis,
                yaxis = yaxis,
                ticks = [],
                guidefontsize = 7,
                framestyle = :box,
                size = (200, 150),
                background_color=:transparent, foreground_color=:black,
                kwargs...
    )


    for k = 1:n_groups
        v = groups[k]
        scatter!(y[1,v], y[2,v], 
                markersize = markersize,
                markerstrokewidth = markerstrokewidth,
                markershape = markershapes[k],
                c = c[k],
                labels = label_prefix * string(k),
                legendfontsize = legendfontsize
                )     
    end
    return p

end

In [ ]:
p = plot_dim_red2(embedding, cluster_indices; 
                    dim_red = "UMAP", 
                    xaxis = "UMAP-1", 
                    yaxis = "UMAP-2",
                    xlims = (-9, 20)
                    )
savefig("analysis/ECM/combined/hierarchical_clustering_UMAP.svg")

In [ ]:
# plot selected examples
# select four random examples
cluster_4 = Dict(i => sample(cluster_indices[i], 4, replace = :false) for i in keys(cluster_indices))

#save("analysis/ECM/combined/hierarchical_cluster_examples.jld2", Dict(string(k) => v for (k,v) in cluster_4))
cluster_4 = load("analysis/ECM/combined/hierarchical_cluster_examples.jld2")
cluster_4 = Dict(parse(Int32, k) => v for (k,v) in cluster_4)

figname = "analysis/ECM/combined/hierarchical_clusters_representatives.png"
plot_ECM_PSRH(cluster_4, idx_ROI, figname)

In [ ]:
#save("analysis/ECM/combined/hierarchical_clustering.jld2", "cluster_indices", cluster_indices)

In [ ]:
cluster_indices = load("analysis/ECM/combined/hierarchical_clustering.jld2", "cluster_indices")

In [ ]:
p = plot_dim_red(embedding, cluster_indices; 
                    dim_red = "UMAP", 
                    xaxis = "UMAP-1", 
                    yaxis = "UMAP-2",
                    xlims = (-12, 20)
                    )

# Plots for figures (using hierarchical clustering)

In [ ]:
# load clusters
cluster_indices = load("analysis/ECM/combined/hierarchical_clustering.jld2")["cluster_indices"]

# plot clusters in 2 dimensions
embedding = Array(CSV.read("analysis/ECM/combined/umap.csv", header = false))
p = plot_dim_red(embedding, cluster_indices; 
                    dim_red = "UMAP", 
                    xaxis = "UMAP-1", 
                    yaxis = "UMAP-2",
                    xlims = (-10, 14)
                    )
#savefig("analysis/ECM/combined/umap_clusters.svg")

In [ ]:
# plot one representative sample in each cluster
# load clusters
cluster_4 = load("analysis/ECM/combined/hierarchical_cluster_examples.jld2")
cluster_4 = Dict(parse(Int32, k) => v for (k,v) in cluster_4)

In [ ]:
cluster_rep = Dict(1 => [372],
    2 => [121],
    3 => [371],
    4 => [304],
    5 => [171],
    6 => [276],
    7 => [238],
    8 => [288],
    9 => [147]
)

In [ ]:
figname = "analysis/ECM/combined/umap_clusters_representatives_figure_highres.png"
plot_ECM_PSRH(cluster_rep, idx_ROI, figname; grid_layout = grid(3,6), size = (6000, 4200), bottom_margin = 8mm, right_margin = 40mm)

# Compute distances between clusters 

In [ ]:
# load the cluster index (new clusters)
df = DataFrame(CSV.read("analysis/ECM/combined/cluster_labels_python.csv", header = false ))
clusters_original = Dict(-1 => [], 0 => [], 1 => [], 2 => [], 3 => [], 4 => [], 5 =>[], 6 => [], 7 => [])
for i = 1:size(df,1)
    cluster = Int(df[i, :Column1])
    push!(clusters_original[cluster], i)
end

# load features array
features_array = load("analysis/ECM/combined/features.jld2")["features"];

In [ ]:
n_clusters = length(clusters_original)
cluster_distances = zeros((n_clusters, n_clusters))
mean_features = Dict(i => mean(features_array[:, clusters_original[i]], dims = 2) for i = -1:n_clusters - 2)


In [ ]:
for i =1:n_clusters
    for j =1:n_clusters
        cluster_distances[i,j] = cluster_distances[j,i] = Euclidean()(mean_features[i-2], mean_features[j-2])
    end
end

In [ ]:
heatmap(cluster_distances)

In [ ]:
cluster_distances

In [ ]:
writedlm("analysis/ECM/combined/cluster_TDA_distances_from_mean_features.csv",  cluster_distances, ',')

In [ ]:
heatmap(cluster_distances, yflip = :true, ticks = [], aspect_ratio = :equal, logscale = true, size = (600, 500), axis = false)
savefig("analysis/ECM/combined/UMAP_cluster_distances.svg")

Compute distances by average distance between pairs belonging to those clsuters


In [ ]:
for i =-1:n_clusters-2
    for j =-1:n_clusters-2
        all_distances = []
        for ROI_i in clusters_original[i]
            for ROI_j in clusters_original[j]
                d = Euclidean()(features_array[:,ROI_i], features_array[:, ROI_j])
                push!(all_distances, d)
            end
        end
        cluster_distances[i+2,j+2] = cluster_distances[j+2,i+2] = mean(all_distances)
    end
end

In [ ]:
heatmap(cluster_distances)

In [ ]:
cluster_distances

In [ ]:
heatmap(cluster_distances, yflip = :true, ticks = [], aspect_ratio = :equal, logscale = true, size = (600, 500), axis = false)
savefig("analysis/ECM/combined/UMAP_cluster_distances_from_mean_of_pairwise_distances.svg")

In [ ]:
writedlm("analysis/ECM/combined/cluster_TDA_distances_from_pairwise_distances.csv",  cluster_distances, ',')

# UMAP - dimension 0

In [ ]:
# get ranges of x and y (useful for plotting )
xmin = PI0.xs[1]
xmax = PI0.xs[end]

ymin = PI0.ys[1]
ymax = PI0.ys[end];

In [ ]:
# index and ROIs 
#ROIs = collect(keys(ECM_PI0))
#idx_ROI = Dict(i => roi for (i, roi) in enumerate(ROIs));

#save("analysis/ECM/dim_0/idx_ROI.jld2", "idx_ROI", idx_ROI)
idx_ROI = load("analysis/ECM/dim_0/idx_ROI.jld2")["idx_ROI"];

In [ ]:
function create_features_array_dim0(PI_dict, idx_ROI)
    n = length(idx_ROI)
    features_array = hcat([PI_dict[idx_ROI[i]] for i = 1:n]...)
    println("features array shape: ", size(features_array))

    features_centered = features_array .- mean(features_array, dims = 2);
    return features_array, features_centered
end

function create_features_array_dim1(PI_dict, idx_ROI)
    n = length(idx_ROI)
    features_array = hcat([vec(PI_dict[idx_ROI[i]]) for i = 1:n]...)
    println("features array shape: ", size(features_array))

    features_centered = features_array .- mean(features_array, dims = 2);
    return features_array, features_centered
end


In [ ]:
features_array, features_centered = create_features_array_dim0(ECM_PI0, idx_ROI);

In [ ]:
#save("analysis/ECM/dim_0/features.jld2", "features", features_centered)

In [ ]:
Random.seed!(10)
embedding = umap(features_centered, 2; n_neighbors = 5)
#writedlm("analysis/ECM/dim_0/umap.csv", embedding, ",")


In [ ]:
# load
embedding = Array(CSV.read("analysis/ECM/dim_0/umap.csv", header = false))
gr()
n = size(embedding, 2)
p = scatter(embedding[1,:], embedding[2,:], 
        markercolor = "slategrey",
        markersize = 5, 
        label = "", 
        xticks = [], 
        yticks = [], 
        framestyle = :box,  
        xlabel = "UMAP-1",
        ylabel = "UMAP-2",
        guidefontsize = 15,
        leftmargin = 5mm,
        size = (450, 350),
        hover = 1:n,
        legend = :topright)
savefig("analysis/ECM/dim_0/umap.pdf")
plot(p)

# UMAP - dimension 1

In [ ]:
# index and ROIs 
#ROIs = collect(keys(ECM_PI1))
#idx_ROI = Dict(i => roi for (i, roi) in enumerate(ROIs));

#save("analysis/ECM/dim_1/idx_ROI.jld2", "idx_ROI", idx_ROI)
idx_ROI = load("analysis/ECM/dim_1/idx_ROI.jld2")["idx_ROI"];

In [ ]:
features_array, features_centered = create_features_array_dim1(ECM_PI1, idx_ROI);

In [ ]:
save("analysis/ECM/dim_1/features.jld2", "features", features_array)

In [ ]:
Random.seed!(10)
embedding = umap(features_centered, 2; n_neighbors = 5)
#writedlm("analysis/ECM/dim_1/umap.csv", embedding, ",")


In [ ]:
# load
embedding = Array(CSV.read("analysis/ECM/dim_1/umap.csv", header = false))
gr()
n = size(embedding, 2)
p = scatter(embedding[1,:], embedding[2,:], 
        markercolor = "slategrey",
        markersize = 5, 
        label = "", 
        xticks = [], 
        yticks = [], 
        framestyle = :box,  
        xlabel = "UMAP-1",
        ylabel = "UMAP-2",
        guidefontsize = 15,
        leftmargin = 5mm,
        size = (450, 350),
        hover = 1:n,
        legend = :topright)
savefig("analysis/ECM/dim_1/umap.pdf")
plot(p)

# PCA dimension 1

In [ ]:
idx_ROI = load("analysis/ECM/dim_1/idx_ROI.jld2")["idx_ROI"];
PI1_new = Dict(i => ECM_PI1[idx_ROI[i]] for i = 1:length(idx_ROI));

In [ ]:
# compute PCA
transformed, eigenvectors, variance_1, variance_2, _ = PI_to_PCA2(PI1_new; pratio = 0.99)

#save("analysis/ECM/dim_1/PCA.jld2",
#    "transformed", transformed,
#    "eigenvectors", eigenvectors,
#    "variance_1", variance_1,
#    "variance_2", variance_2)

println("number of components: ", length(eigenvectors))
println("variance explained by 1 eigenvectors: ", variance_1)
println("variance explained by 2 eigenvectors: ", variance_2)
println("variance difference between 2 and 1:", variance_2 - variance_1)

In [ ]:
df_pca1 = convert(DataFrame, Array(Transpose(transformed)))
col_names = ["pca_coord_" * string(i) for i = 1:17];
idx_ROI_list = [idx_ROI[i] for i = 1:396];
rename!(df_pca1, col_names)
df_pca1[:, :idx_ROI] = idx_ROI_list;
#CSV.write("analysis/ECM/dim_1/PCA_coord.csv", df_pca1)

In [ ]:
# plot with regions
gr()

y = transformed
markersize = 3
legendfontsize = 5
n = size(y, 2)
p = scatter(y[1,:], y[2,:], 
        markercolor = "lightgray",
        alpha = 0.6,
        markersize = markersize, 
        markerstrokewidth = 3,
        label = "", 
        xaxis = "PC1 (88%)",
        yaxis = "PC2 (7%)",
        #xtickfontsize = 15,
        #ytickfontsize = 15,
        #xrotation = 45,     
        xticks = (0, 0),
        yticks = (0,0),
        guidefontsize = 7,
        framestyle = :box,
        size = (200, 150),
        #leftmargin = 2mm,
        legend = :bottomleft
        #background_color=:transparent, foreground_color=:black,
        )
#scatter!(y[1,R1], y[2,R1], label = "", markersize = markersize, markershape = :rect, markercolor = c1, labels = "R1", legendfontsize = legendfontsize)
#scatter!(y[1,R2], y[2,R2], label = "", markersize = markersize, markershape = :utriangle, markercolor = c2, labels = "R2")
#scatter!(y[1,R3], y[2,R3], label = "", markersize = markersize, markershape = :star, markercolor = c3, labels = "R3")
#scatter!(y[1,R4], y[2,R4], label = "", markersize = markersize, markershape = :diamond, markercolor = c4, labels = "R4")
#savefig("analysis/ECM/dim_1/pca.pdf")
plot(p)

In [ ]:
# plot with regions
#R1 =  [290, 152, 54]
#R2 = [117, 279, 237]
#R3 = [176, 139, 159]
#R4 = [162, 111,105];

R1 =  [290]
R2 = [117]
R3 = [176]
R4 = [162];

gr()

markersize = 3
legendfontsize = 5
n = size(y, 2)
annotated = vcat(R1, R2, R3, R4)
nonannotated = [i for i = 1:size(y, 2) if i ∉ annotated]
p = scatter(y[1,nonannotated], y[2,nonannotated], 
        markercolor = "lightgray",
        alpha = 0.6,
        markersize = markersize, 
        markerstrokewidth = 3,
        label = "", 
        xaxis = "PC1 (83%)",
        yaxis = "PC2 (8%)",
        #xtickfontsize = 15,
        #ytickfontsize = 15,
        #xrotation = 45,     
        xticks = (0, 0),
        yticks = (0,0),
        guidefontsize = 7,
        framestyle = :box,
        size = (200, 150),
        #leftmargin = 2mm,
        legend = :topleft
        #background_color=:transparent, foreground_color=:black,
        )
scatter!(y[1,R1], y[2,R1], label = "", markersize = markersize, markershape = :rect, markercolor = c1, labels = "R1", legendfontsize = legendfontsize)
scatter!(y[1,R2], y[2,R2], label = "", markersize = markersize, markershape = :utriangle, markercolor = c2, labels = "R2")
scatter!(y[1,R3], y[2,R3], label = "", markersize = markersize, markershape = :star, markercolor = c3, labels = "R3")
scatter!(y[1,R4], y[2,R4], label = "", markersize = markersize, markershape = :diamond, markercolor = c4, labels = "R4")
savefig("analysis/ECM/dim_1/pca.pdf")
plot(p)

In [ ]:
# get min and max pixels of the first four eigenvectors
eigenvector_min = minimum(minimum.(eigenvectors[i] for i = 1:4))
eigenvector_max = maximum(maximum.(eigenvectors[i] for i = 1:4))

println("min pixel: ", eigenvector_min)
println("max pixel: ", eigenvector_max)

In [ ]:
function plot_PI2(PI, x_min, x_max, y_min, y_max; 
    kwargs...) 

    n = size(PI,1)
    xinterval = (x_max - x_min)/4
    yinterval = (y_max - y_min)/4
    p = heatmap(PI, 
    label = "",
    title = "",
    framestyle = :box,
    xticks = (5:5:20, Int32.(round.(x_min+xinterval:xinterval:x_max))),
    yticks = (5:5:20, Int32.(round.(y_min+yinterval:yinterval:y_max))),
    ;kwargs...)

    return p
end

In [ ]:
# plot the first four eigenvectors
plot_scale = 20 # only show plot_scale% of persistence image
ps = [plot_PI2(eigenvectors[i], PI1_xmin, PI1_xmax, PI1_ymin, PI1_ymax, 
            clims = (eigenvector_min, eigenvector_max), 
            xlabel = "birth",
            ylabel = "persistence",
            show_axis = false,
            left_margin = 5mm,
            bottom_margin = 5mm,
            xrotation = 45,
            #framestyle = :box,
            #title = "eigenvector " * string(i), 
            legend = :false # no colorbar 
            ) for i =1:2]

l = @layout[grid(1,2) a{0.05w}] # Stack a layout that rightmost one is for color bar
Plots.GridLayout(1, 2)

n = 100 # length of colorbar (as a vector)
cbar_interval = 0.2
cbar_ticks = vcat(reverse(collect(0:cbar_interval: -eigenvector_min))[1:end-1] .* -1, collect(0:cbar_interval:eigenvector_max))
cbar_loc = [cbar_tickvals_to_loc(eigenvector_min, eigenvector_max, n, val) for val in cbar_ticks]

p = plot(ps..., 
         heatmap(collect(range(eigenvector_min, eigenvector_max, length = n)) .* ones(n,1), 
                legend=:none, 
                xticks=:none,
                yticks=(cbar_loc, cbar_ticks)),
         layout=l,
         topmargin = 3mm,
         size = (500, 200))
#savefig("analysis/ECM/dim_1/eigenvectors.svg")
plot(p)

In [ ]:
# load PD_max
PD_max = load("data/4000x4000_combined/ECM_PD/PD_max.jld2")
max0 = PD_max["max0"]
max1 = PD_max["max1"];

In [ ]:
regions = Dict("R1" => R1[1], "R2"=> R2[1], "R3"=> R3[1], "R4"=> R4[1])

plot_array = []

# get maximum PI pixel value for the selected examples
PI_pixel_max = maximum([maximum(ECM_PI1[idx_ROI[i]]) for i in values(regions)])
for j = 1: 4
    R_string = "R" * string(j)
    R = regions[R_string]
    for i in R
        f = idx_ROI[i]

        # plot ECM
        p_ECM = Images.load("data/4000x4000_combined/subregion_ECM/" * f * ".tif" )
        plot(p_ECM, ticks = [], frame = :box, size = (200,200))
        savefig("analysis/ECM/dim_1/PCA_coordinates_" * R_string * "_" * string(i) * "_ECM.png")
        push!(plot_array, plot(p_ECM, ticks = [], frame = :box))

        p_PD = plot_PD(PD1[f], pd_min = 0, pd_max = 1600, lw = 1, 
            tickfontsize = 9,
            labelfontsize = 12,
            markersize = 3,
            ticks = [0, 400, 800, 1200, 1600],
            xlabel = "birth", ylabel = "death", 
            size = (200,200), 
            xrotation = 45,
            leftmargin = -3mm, 
            bottommargin = -5mm
            )
        savefig("analysis/ECM/dim_1/PCA_coordinates_" * R_string * "_" * string(i) * "_PD.svg")
        push!(plot_array, p_PD)

        p_PI = plot_PI2(ECM_PI1[f], PI1_xmin, PI1_xmax, PI1_ymin, PI1_ymax; 
                xlabel = "birth",
                ylabel = "persistence",
                xrotation = 45,
                show_axis = false,
                left_margin = -1mm,
                bottom_margin = 0mm,
                legend = :none,
            tickfontsize = 10,
            size = (200,200))
        savefig("analysis/ECM/dim_1/PCA_coordinates_" * R_string * "_" * string(i) * "_PI.svg")
        push!(plot_array, p_PI)

        p_coords = plot_scores(transformed[1:4,i]; xtickfontsize = 8, ytickfontsize = 8, coord_label = "PC")
        p_coords = plot(p_coords, size = (200, 200))
        savefig("analysis/ECM/dim_1/PCA_coordinates_" * R_string * "_" * string(i) * "_coords.svg")
        push!(plot_array, p_coords)
    end
end
p = plot(plot_array..., layout = grid(4,4, widths=[0.21 ,0.21, 0.37, 0.21]), size = (1000, 900) )
savefig("analysis/ECM/dim_1/PCA_coordinates.png")